# Import Library






In [48]:
import matplotlib.pyplot as plt
import seaborn as sns
import string
import datetime
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

from xgboost import XGBClassifier
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder
from sklearn import preprocessing
from PIL import Image
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE

# Read Data

In [49]:
train_work = pd.read_csv('./sample_data/work_train.csv')
test_work = pd.read_csv('./sample_data/work_test.csv')
train_info = pd.read_csv('./sample_data/info_train.csv')
test_info = pd.read_csv('./sample_data/info_test.csv')
y_train = pd.read_csv('./sample_data/label_train.csv')

# Preprocessing

**Fill the missing data, 
change from uppercase to lowercase, and delete punctuation**

In [50]:
assert 'NA' not in train_work['job/role'].values
assert 'NA' not in test_work['job/role'].values

train_work['job/role'].fillna('NA', inplace=True)
test_work['job/role'].fillna('NA', inplace=True)

train_work['job/role'] = train_work['job/role'].str.lower()
test_work['job/role'] = test_work['job/role'].str.lower()

In [51]:
assert 'NA' not in train_work['address'].values
assert 'NA' not in test_work['address'].values

train_work['address'].fillna('NA', inplace=True)
test_work['address'].fillna('NA', inplace=True)

train_work['address'] = train_work['address'].str.lower()
test_work['address'] = test_work['address'].str.lower()

In [52]:
assert 'NA' not in train_info['address'].values
assert 'NA' not in test_info['address'].values

train_info['address'].fillna('NA', inplace=True)
test_info['address'].fillna('NA', inplace=True)

train_info['address'] = train_info['address'].str.lower()
test_info['address'] = test_info['address'].str.lower()

In [53]:
assert 'NA' not in train_work['id_office'].values
assert 'NA' not in test_work['id_office'].values

train_work['id_office'].fillna('NA', inplace=True)
test_work['id_office'].fillna('NA', inplace=True)

train_work['id_office'] = train_work['id_office'].str.lower()
test_work['id_office'] = test_work['id_office'].str.lower()

In [54]:
train_work

,id,id_bh,id_management,id_office,company_type,job/role,from_date,to_date,employee_lv,address
0,1,113039360,106,tf2212f,-1,giám đốc,20130100,20151200,7.0,hà nội
1,1,113039360,106,tf2212f,-1,giám đốc,20160100,20220400,10.0,hà nội
2,2,116074930,102,tb16010,-1,nhân viên lễ tân,20160600,20161200,7.0,hà nội
3,2,116074930,102,tb16010,-1,nhân viên lễ tân,20170100,20170300,8.0,hà nội
4,2,116074930,102,na,-1,na,20170400,20170700,-1.0,na
...,...,...,...,...,...,...,...,...,...,...
247554,55008,2616117553,2600,yn0027z,-1,công nhân đm rôbốt,20100800,20100900,2.0,na
247555,55008,2616117553,2600,na,-1,na,20101000,20161000,-1.0,na
247556,55008,2616117553,2600,tz0085z,1,phụ kho,20161100,20171200,6.0,vĩnh phúc
247557,55008,2616117553,2600,tz0085z,1,phụ kho,20180100,20191200,8.0,vĩnh phúc


In [55]:
def remove_punctuation(txt):
  txt_nopunct = "".join([c for c in txt if c not in string.punctuation])
  
  return txt_nopunct

In [56]:
train_work['address'] = train_work['address'].apply(lambda x: remove_punctuation(x))
test_work['address'] = test_work['address'].apply(lambda x: remove_punctuation(x))
train_info['address'] = train_info['address'].apply(lambda x: remove_punctuation(x))
test_info['address'] = test_info['address'].apply(lambda x: remove_punctuation(x))


In [57]:
train_work['job/role'] = train_work['job/role'].apply(lambda x: remove_punctuation(x))
test_work['job/role'] = test_work['job/role'].apply(lambda x: remove_punctuation(x))

**Change from_date and to_date to datetime type**

In [58]:
def conv_date(date_int):
  date_str = str(date_int)
  y = int(date_str[0:4])
  m = int(date_str[4:6])
  d = 1
  
  return pd.Timestamp(y, m, d)

In [59]:
train_work['from_date'] = train_work.from_date.apply(conv_date)
train_work['to_date'] = train_work.to_date.apply(conv_date)
test_work['from_date'] = test_work.from_date.apply(conv_date)
test_work['to_date'] = test_work.to_date.apply(conv_date)

In [60]:
def diff_months(row):
  from_date = row['from_date']
  to_date = row['to_date']

  return (to_date.to_period('M') - from_date.to_period('M')).n

In [61]:
train_work['elapsed_months'] = train_work.apply(lambda row: diff_months(row), axis=1)
test_work['elapsed_months'] = test_work.apply(lambda row: diff_months(row), axis=1)

**Calculate cumulative_work Duration for each person's work records**

In [62]:
train_work['cumulative_work'] = train_work.groupby(['id_bh'])['elapsed_months'].apply(lambda x: x.cumsum())
test_work['cumulative_work'] = test_work.groupby(['id_bh'])['elapsed_months'].apply(lambda x: x.cumsum())

In [63]:
train_ids = train_info['id_bh']
test_ids = test_info['id_bh']
for id in train_ids:
  works = train_work[train_work['id_bh'] == id]
  if works.to_date.is_monotonic_increasing == False:
    print(f"Not sorted id {id}")
    break

In [64]:
for id in test_ids:
  works = test_work[test_work['id_bh'] == id]
  if works.to_date.is_monotonic_increasing == False:
    print(f"Not sorted id {id}")
    break

**Keep first 2 tokens in the `job/role` field**

In [65]:
def keep_first_two_tokens(value):
  if isinstance(value, str) and len(value.split(" ")) > 2:
    tokens = value.split(" ")
    return tokens[0] + " " + tokens[1]
  else:
    return value

In [66]:
train_work['jobType'] = train_work['job/role'].apply(lambda x: keep_first_two_tokens(x))
test_work['jobType'] = test_work['job/role'].apply(lambda x: keep_first_two_tokens(x))

**Take the last 2 tokens in the `address` field and copy to `**

In [67]:
def keep_last_two_tokens(value):
  if isinstance(value, str) and len(value.split(" ")) > 2:
    tokens = value.split(" ")
    return tokens[-2] + " " + tokens[-1]
  else:
    return value

In [68]:
train_work['addressNor'] = train_work.address.map(lambda x: keep_last_two_tokens(x))
test_work['addressNor'] = test_work.address.map(lambda x: keep_last_two_tokens(x))

train_info['addressNor'] = train_info.address.map(lambda x: keep_last_two_tokens(x))
test_info['addressNor'] = test_info.address.map(lambda x: keep_last_two_tokens(x))

**Join tables**

In [69]:
train_merged = pd.merge(train_work, train_info, on='id_bh', how="inner")
test_merged = pd.merge(test_work, test_info, on='id_bh', how="inner")
train_merged.head()

,id,id_bh,id_management,id_office,company_type,job/role,from_date,to_date,employee_lv,address_x,elapsed_months,cumulative_work,jobType,addressNor_x,bithYear,gender,address_y,addressNor_y
0,1,113039360,106,tf2212f,-1,giám đốc,2013-01-01,2015-12-01,7.0,hà nội,35,35,giám đốc,hà nội,1971,MALE,hà nội,hà nội
1,1,113039360,106,tf2212f,-1,giám đốc,2016-01-01,2022-04-01,10.0,hà nội,75,110,giám đốc,hà nội,1971,MALE,hà nội,hà nội
2,2,116074930,102,tb16010,-1,nhân viên lễ tân,2016-06-01,2016-12-01,7.0,hà nội,6,6,nhân viên,hà nội,1993,FEMALE,thành phố hà nội,hà nội
3,2,116074930,102,tb16010,-1,nhân viên lễ tân,2017-01-01,2017-03-01,8.0,hà nội,2,8,nhân viên,hà nội,1993,FEMALE,thành phố hà nội,hà nội
4,2,116074930,102,na,-1,na,2017-04-01,2017-07-01,-1.0,na,3,11,na,na,1993,FEMALE,thành phố hà nội,hà nội


In [70]:
train_merged = pd.merge(train_merged, y_train, on='id_bh', how="inner")

**Merge Work Tables**

In [71]:
train_merged['train_or_test'] = 'train'
test_merged['train_or_test'] = 'test'

In [72]:
print(f"{len(train_merged)} + {len(test_merged)} = {len(train_merged) + len(test_merged)}")

247559 + 162283 = 409842


In [73]:
merged_work = pd.merge(left=train_merged, right=test_merged, how='outer')

In [74]:
label_encoder = preprocessing.LabelEncoder()

**Encode**

In [75]:
merged_work['job_encoded'] = label_encoder.fit_transform(merged_work['job/role'])

In [76]:
merged_work['io_encoded'] = label_encoder.fit_transform(merged_work['id_office'])
merged_work['addx_encoded'] = label_encoder.fit_transform(merged_work['address_x'])
merged_work['jobtype_encoded'] = label_encoder.fit_transform(merged_work['jobType'])
merged_work['addnorx_encoded'] = label_encoder.fit_transform(merged_work['addressNor_x'])
merged_work['addy_encoded'] = label_encoder.fit_transform(merged_work['address_y'])
merged_work['addnory_encoded'] = label_encoder.fit_transform(merged_work['addressNor_y'])

In [77]:
merged_work.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 409842 entries, 0 to 409841
Data columns (total 27 columns):
 #   Column           Non-Null Count   Dtype         
---  ------           --------------   -----         
 0   id               409842 non-null  int64         
 1   id_bh            409842 non-null  int64         
 2   id_management    409842 non-null  int64         
 3   id_office        409842 non-null  object        
 4   company_type     409842 non-null  int64         
 5   job/role         409842 non-null  object        
 6   from_date        409842 non-null  datetime64[ns]
 7   to_date          409842 non-null  datetime64[ns]
 8   employee_lv      409834 non-null  float64       
 9   address_x        409842 non-null  object        
 10  elapsed_months   409842 non-null  int64         
 11  cumulative_work  409842 non-null  int64         
 12  jobType          409842 non-null  object        
 13  addressNor_x     409842 non-null  object        
 14  bithYear         409

In [78]:
merged_work_shortcut = merged_work.drop(['id_office', 'job/role', 'from_date', 'to_date', 'address_x', 'address_y', 'elapsed_months', 'jobType', 'addressNor_x', 'addressNor_y' ], axis = 1)

In [79]:
merged_work_shortcut

,id,id_bh,id_management,company_type,employee_lv,cumulative_work,bithYear,gender,label,train_or_test,job_encoded,io_encoded,addx_encoded,jobtype_encoded,addnorx_encoded,addy_encoded,addnory_encoded
0,1,113039360,106,-1,7.0,35,1971,MALE,4.0,train,15122,10173,408,1160,219,397,385
1,1,113039360,106,-1,10.0,110,1971,MALE,4.0,train,15122,10173,408,1160,219,397,385
2,2,116074930,102,-1,7.0,6,1993,FEMALE,2.0,train,22938,7991,408,1863,219,715,385
3,2,116074930,102,-1,8.0,8,1993,FEMALE,2.0,train,22938,7991,408,1863,219,715,385
4,2,116074930,102,-1,-1.0,11,1993,FEMALE,2.0,train,20531,4424,583,1787,347,715,385
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
409837,54995,2615002452,2600,-1,8.0,53,1984,FEMALE,NaN,test,9186,16137,1377,853,775,957,760
409838,54995,2615002452,2600,-1,9.0,74,1984,FEMALE,NaN,test,9186,16137,1377,853,775,957,760
409839,54995,2615002452,2600,-1,10.0,79,1984,FEMALE,NaN,test,9186,16137,1377,853,775,957,760
409840,55003,2621579999,2600,6,24.0,0,1984,MALE,NaN,test,15165,3853,1304,1160,775,508,496


In [80]:
train_work = merged_work_shortcut[merged_work['train_or_test'] == 'train']
test_work = merged_work_shortcut[merged_work['train_or_test'] == 'test']

train_work.drop(columns=['train_or_test'], inplace=True)
test_work.drop(columns=['train_or_test'], inplace=True)

C:\Users\Kyoraku\AppData\Local\Temp\ipykernel_5788\1968491522.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_work.drop(columns=['train_or_test'], inplace=True)
C:\Users\Kyoraku\AppData\Local\Temp\ipykernel_5788\1968491522.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_work.drop(columns=['train_or_test'], inplace=True)


In [81]:
test_work = test_work.reset_index()

In [82]:
test_work.drop(columns=['label'], inplace = True)

In [83]:
test_work.drop(columns=['index'], inplace = True)

In [84]:
train_work = train_work.drop_duplicates(subset=['id'], keep='last')
test_work = test_work.drop_duplicates(subset=['id'], keep='last')

In [85]:
train_work.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 27502 entries, 1 to 247558
Data columns (total 16 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   id               27502 non-null  int64  
 1   id_bh            27502 non-null  int64  
 2   id_management    27502 non-null  int64  
 3   company_type     27502 non-null  int64  
 4   employee_lv      27502 non-null  float64
 5   cumulative_work  27502 non-null  int64  
 6   bithYear         27502 non-null  int64  
 7   gender           27502 non-null  object 
 8   label            27502 non-null  float64
 9   job_encoded      27502 non-null  int32  
 10  io_encoded       27502 non-null  int32  
 11  addx_encoded     27502 non-null  int32  
 12  jobtype_encoded  27502 non-null  int32  
 13  addnorx_encoded  27502 non-null  int32  
 14  addy_encoded     27502 non-null  int32  
 15  addnory_encoded  27502 non-null  int32  
dtypes: float64(2), int32(7), int64(6), object(1)
memory usage

In [86]:
train_work['label'] = train_work['label'].astype(np.int64)

In [87]:
vec = DictVectorizer()
train_data = train_work[['job_encoded','jobtype_encoded', 'employee_lv', 'bithYear','gender', 'company_type','cumulative_work', 'id_management', 'io_encoded',
                         'addx_encoded', 'addnorx_encoded', 'addy_encoded', 'addnory_encoded']].to_dict('records')
X_train = vec.fit_transform(train_data).toarray()
print(X_train[1])

[ 2.1900e+02  3.8500e+02  4.0800e+02  7.1500e+02  1.9930e+03 -1.0000e+00
  4.9000e+01  9.0000e+00  1.0000e+00  0.0000e+00  1.0200e+02  7.9910e+03
  2.3967e+04  1.8630e+03]


In [88]:
test_data = test_work[['job_encoded','jobtype_encoded', 'employee_lv', 'bithYear','gender', 'company_type','cumulative_work', 'id_management', 'io_encoded',
                         'addx_encoded', 'addnorx_encoded', 'addy_encoded', 'addnory_encoded']].to_dict('records')
test_id = test_work['id_bh'].tolist()
X_test = vec.transform(test_data).toarray()
print(X_test[1])

[ 2.1900e+02  4.9800e+02  1.0710e+03  5.1000e+02  1.9710e+03 -1.0000e+00
  4.8000e+01  9.0000e+00  0.0000e+00  1.0000e+00  1.0200e+02  7.9800e+03
  2.0665e+04  1.8630e+03]


In [89]:
Y_train = train_work['label']

In [ ]:
multi_model = XGBClassifier()
# Add silent=True to avoid printing out updates with each cycle
multi_model.fit(X_train, Y_train, verbose=False)

In [ ]:
prediction = multi_model.predict(X_test)

In [ ]:
output = pd.DataFrame({'id_bh': test_id, 'label': prediction})
output.to_csv('prediction_test.csv', index=False)